<a href="https://colab.research.google.com/github/SourCherries/VAE-GAN-CelebA/blob/master/visualize_celeba_15_AFC_sheldon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/SourCherries/VAE-GAN-CelebA.git

%tensorflow_version 1.x

import sys, getopt
if sys.version_info < (3,4):
    print('You are running an older version of Python!\n\n',
          'You should consider updating to Python 3.4.0 or',
          'higher.\n')
print(sys.version)

sys.path.insert(0,'/content/VAE-GAN-CelebA/libs')
sys.path.insert(0,'/content/VAE-GAN-CelebA/')

import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import libs.make_network as make_network
import libs.utils as utils
from scipy.io import loadmat, savemat
import imageio
from skimage.util import montage
import pandas as pd
import seaborn as sns

import zipfile
import io
from PIL import Image
from torchvision import transforms

print('TF version = ',tf.__version__)

# Download Tensorflow checkpoint
! gdown --id 1o9nUhtZEX9GZXsTmvCXSNjEjDQ7Q1wHV

# CelebA
# https://drive.google.com/file/d/172v_Sfx-MyE2nS59jxl6W9-so3Xvyhkj/view?usp=sharing
! gdown --id 172v_Sfx-MyE2nS59jxl6W9-so3Xvyhkj



Cloning into 'VAE-GAN-CelebA'...
remote: Enumerating objects: 193, done.
remote: Counting objects: 100% (121/121), done.
remote: Compressing objects: 100% (71/71), done.
remote: Total 193 (delta 58), reused 106 (delta 49), pack-reused 72
Receiving objects: 100% (193/193), 4.66 MiB | 27.12 MiB/s, done.
Resolving deltas: 100% (87/87), done.
After that, `%tensorflow_version 1.x` will throw an error.

Your notebook should be updated to use Tensorflow 2.
See the guide at https://www.tensorflow.org/guide/migrate#migrate-from-tensorflow-1x-to-tensorflow-2.

TensorFlow 1.x selected.
3.7.13 (default, Apr 24 2022, 01:04:09) 
[GCC 7.5.0]
TF version =  1.15.2
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1o9nUhtZEX9GZXsTmvCXSNjEjDQ7Q1wHV
To: /content/vaegan_celeba.ckpt
10

## Model

In [2]:
sess, X, G, Z, Z_mu, is_training, saver = make_network.make_network()

if os.path.exists("vaegan_celeba.ckpt"):
    saver.restore(sess, "vaegan_celeba.ckpt")
    print("VAE-GAN model restored.")
else:
    print("Pre-trained network appears to be missing.")
    sys.exit()






encoder/input: [None, 128, 128, 3]

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

layer: 0 , shape: [None, 64, 64, 192]
layer: 1 , shape: [None, 32, 32, 256]
layer: 2 , shape: [None, 16, 16, 384]
layer: 3 , shape: [None, 8, 8, 512]
layer: 4 , shape: [None, 4, 4, 768]


Instructions for updating:
Use `tf.global_variables_initializer` instead.

Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Restoring parameters from vaegan_celeba.ckpt
VAE-GAN model restored.


## Mean Z for 15 identities
Final is Sheldon, first 7 are most similar to Sheldon, and next 7 have mean Z that are orthogonal to Sheldon.

In [7]:
z_file = "/content/VAE-GAN-CelebA/celeba-15-AFC-sheldon-last"
Z = np.fromfile(z_file)
Z = Z.reshape(15, 1025)
ID = Z[:, 0]
Z = Z[:, 1:]

## Batchwise convert to images

In [11]:
print(Z.shape)
print(Z.dtype)

(15, 1024)
float64


In [12]:
g = sess.run(G, feed_dict={Z: (1*Z), is_training: False})


def imdeprocess(g):
    stretch = 1.0
    for i in range(g.shape[0]):
        g[i]=np.clip(stretch*g[i] / (g.max()),0,1)
    return g


for gg in g:
  img = imdeprocess(gg)
  plt.imshow(img); plt.show()

TypeError: ignored